<a href="https://colab.research.google.com/github/mariel/Sisteme-Inteligente/blob/Proiect2/proiect2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from torch.utils.data import Dataset, DataLoader
from torch import nn, from_numpy, optim
import numpy as np
#pandas- librărie pentru lucrul cu fișierele
import pandas as pd
import torch
import torch.nn as nn

In [0]:
df=pd.read_csv("/content/pd_speech_features.csv",header=None)

In [27]:
df.values

array([[nan, nan, 'Baseline Features', ..., nan, nan, nan],
       ['id', 'gender', 'PPE', ..., 'tqwt_kurtosisValue_dec_35',
        'tqwt_kurtosisValue_dec_36', 'class'],
       ['0', '1', '0.85247', ..., '3.0004', '18.9405', '1'],
       ...,
       ['251', '0', '0.88389', ..., '3.3545', '5.0424', '0'],
       ['251', '0', '0.83782', ..., '2.8332', '3.7131', '0'],
       ['251', '0', '0.81304', ..., '2.6217', '3.1527', '0']],
      dtype=object)

In [0]:
class ParkinsonsDataset(Dataset):
    # Initialize your data, download, etc.
    def __init__(self):
        #Citim setul de date
        df=pd.read_csv("/content/pd_speech_features.csv",header=1, dtype=np.float32)
        xy = torch.from_numpy(df.values)
        self.len = xy.shape[0]
        #Vom folosi ca input toate valorile mai puțin ultima coloană
        self.x_data = xy[:, 0:-1]
        #Vom folosi ca output ultima coloană
        self.y_data = xy[:, [-1]]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [0]:
dataset = ParkinsonsDataset()
#DataLoader - un utilitar ce ne ajută să împărțim setul de date pe batch-uri și astfel să facem antrenare în mod Mini-Batch
train_loader = DataLoader(dataset=dataset,
                          batch_size=16,
                          shuffle=True,
                          num_workers=1)

In [166]:
dataset[0]

(tensor([ 0.0000e+00,  1.0000e+00,  8.5247e-01,  7.1826e-01,  5.7227e-01,
          2.4000e+02,  2.3900e+02,  8.0635e-03,  8.6800e-05,  2.1800e-03,
          1.7600e-05,  6.7000e-04,  1.2900e-03,  2.0000e-03,  5.8830e-02,
          5.1700e-01,  3.0110e-02,  3.4960e-02,  4.8280e-02,  9.0340e-02,
          9.7080e-01,  3.6223e-02,  1.8995e+01,  6.9997e+01,  7.6088e+01,
          7.2466e+01,  5.3934e+02,  1.0318e+03,  2.4472e+03,  3.6551e+03,
          1.0109e+02,  8.3147e+01,  2.5521e+02,  3.9664e+02,  7.7778e-01,
          1.1724e+01,  2.8277e+00,  1.1730e+00,  2.6512e-01,  8.3127e-02,
          1.2004e+06,  1.5347e+00,  3.0152e+00,  4.6263e-04,  2.5997e-04,
          8.3621e-01,  2.0961e+02,  4.5597e+02,  1.2825e+00,  1.3305e+00,
          5.1684e+01,  5.7784e+00,  2.3261e+01,  2.6850e-01,  5.8573e+00,
          2.0023e-01,  9.1817e+00,  1.1428e+01,  8.4781e+00,  2.4874e+00,
          2.2920e-02,  5.5527e-01, -1.6334e+00, -1.5144e+00,  3.9725e-01,
          6.1775e-01,  9.1821e-02,  5.

In [0]:
class Model(nn.Module):

    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.l1 = nn.Linear(754, 384)
        self.l2 = nn.Linear(384,128)
        self.l3 = nn.Linear(128, 96)
        self.l4 = nn.Linear(96,32)
        self.l5 = nn.Linear(32,24)
        self.l6 = nn.Linear(24,8)
        self.l7 = nn.Linear(8,6)
        self.l8 = nn.Linear(6,2)
        self.l9 = nn.Linear(2,1)

        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        out3 = self.sigmoid(self.l3(out2))
        out4 = self.sigmoid(self.l4(out3))
        out5 = self.sigmoid(self.l5(out4))
        out6 = self.sigmoid(self.l6(out5))
        out7 = self.sigmoid(self.l7(out6))
        out8 = self.sigmoid(self.l8(out7))
        y_pred = self.sigmoid(self.l9(out8))
        return y_pred

In [0]:
model = Model()

In [0]:
criterion = nn.BCELoss(reduction='sum')
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [181]:
# Training loop
for epoch in range(200):
  for i, data in enumerate(train_loader, 0):
      # get the inputs
      inputs, labels = data

      # Forward pass: Compute predicted y by passing x to the model
      y_pred = model.forward(inputs)

      # Compute and print loss
      loss = criterion(y_pred, labels)
      print(f'Epoch {epoch + 1} | Batch: {i+1} | Loss: {loss.item():.4f}')

      # Zero gradients, perform a backward pass, and update the weights.
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

Streaming output truncated to the last 5000 lines.
Epoch 96 | Batch: 42 | Loss: 10.0144
Epoch 96 | Batch: 43 | Loss: 8.1078
Epoch 96 | Batch: 44 | Loss: 7.9129
Epoch 96 | Batch: 45 | Loss: 9.0095
Epoch 96 | Batch: 46 | Loss: 6.6781
Epoch 96 | Batch: 47 | Loss: 11.7733
Epoch 96 | Batch: 48 | Loss: 5.4722
Epoch 97 | Batch: 1 | Loss: 10.7125
Epoch 97 | Batch: 2 | Loss: 9.2791
Epoch 97 | Batch: 3 | Loss: 9.9389
Epoch 97 | Batch: 4 | Loss: 7.5144
Epoch 97 | Batch: 5 | Loss: 7.8950
Epoch 97 | Batch: 6 | Loss: 7.8113
Epoch 97 | Batch: 7 | Loss: 12.8567
Epoch 97 | Batch: 8 | Loss: 9.0462
Epoch 97 | Batch: 9 | Loss: 10.9734
Epoch 97 | Batch: 10 | Loss: 9.9385
Epoch 97 | Batch: 11 | Loss: 9.1328
Epoch 97 | Batch: 12 | Loss: 9.0586
Epoch 97 | Batch: 13 | Loss: 9.0258
Epoch 97 | Batch: 14 | Loss: 8.0058
Epoch 97 | Batch: 15 | Loss: 10.1335
Epoch 97 | Batch: 16 | Loss: 7.9828
Epoch 97 | Batch: 17 | Loss: 10.1475
Epoch 97 | Batch: 18 | Loss: 9.0041
Epoch 97 | Batch: 19 | Loss: 5.8434
Epoch 97 | Batc

Un learning rate prea mare duce la o schimbare mare in loss, cateodata este mai mic, altadata mai mare, nu va converge din cauza aceasta, ba chiar sunt mari sanse sa devina divergent.

Un learning rate prea mic duce la pierderea de timp, este ok ca se va ajunge la un rezultat ok, dar timpul pierdut s-ar putea sa fie mult prea mare ca sa se merite.

O retea cu o latime mai mare 
o sa aiba un loss urias (eg: de la 18-20 la 300-1300)


O retea cu mai multe layere o sa aiba un loss din ce in ce mai mic, nu foarte multe ,dar este observabil.

Cu un batch mai mare numarul de batch-uri devine mai mic iar loss-ul mai mare

Pentru bonus: pana acum am incercat sigmoid, tanh si relu; sigmoid da cele mai mici rezultate;
Sigmoid + batch de 64 => rezultate mai mari
Sigmoid + batch de 16 => rezultate mai mici
Nu am observat diferente intre batch de 16 si 8

